<a href="https://colab.research.google.com/github/evroth/Congressional_Analytics/blob/main/Congressional_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This document contais the content for our first project in the Cloud Computing Class.
The analysis below goes attempts to look at the number of starbucks locations in each congressional district, then look at whether the district is red or blue.
We will gather data for Starbucks locations off of Kaggle (Feb 2017). Census data from Census.gov for Total Population, District Number, and State. Lastly data from some source on Political Affiliation, State, and District Number. We will do some EDA and merge the datasets together to perform our analysis.

In [5]:
# starbucks locations
## I am not sure how to always have this data downloaded, might need to re-upload it to colab every time?
import pandas as pd
from geopy.geocoders import Nominatim

# read store location data into a dataframe
store_df = pd.read_csv("starbucks_locations.csv")

# select only US stores
store_df = store_df[store_df['Country'] == "US"]

# Select the "Store Number", "State/Province", "Longitude", and "Latitude" columns
store_df = store_df[["Store Number", "State/Province", "Longitude", "Latitude"]]

# print the dataframe
print(store_df)

       Store Number State/Province  Longitude  Latitude
11964   3513-125945             AK    -149.78     61.21
11965   74352-84449             AK    -149.84     61.14
11966  12449-152385             AK    -149.85     61.11
11967  24936-233524             AK    -149.89     61.13
11968    8973-85630             AK    -149.86     61.14
...             ...            ...        ...       ...
25567   74385-87621             WY    -105.59     41.32
25568   73320-24375             WY    -105.56     41.31
25569  22425-219024             WY    -105.56     41.31
25570  10849-103163             WY    -109.25     41.58
25571  10769-102454             WY    -106.94     44.77

[13608 rows x 4 columns]


In [ ]:
# Install Library
!apt install libspatialindex-dev
!pip install rtree
!pip install geopandas
import geopandas as gpd

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libspatialindex-c6 libspatialindex6
The following NEW packages will be installed:
  libspatialindex-c6 libspatialindex-dev libspatialindex6
0 upgraded, 3 newly installed, 0 to remove and 23 not upgraded.
Need to get 340 kB of archives.
After this operation, 1,538 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 libspatialindex6 amd64 1.9.3-1build1 [266 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/universe amd64 libspatialindex-c6 amd64 1.9.3-1build1 [58.3 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal/universe amd64 libspatialindex-dev amd64 1.9.3-1build1 [16.0 kB]
Fetched 340 kB in 0s (858 kB/s)
Selecting previously unselected package libspatialindex6:amd64.
(Reading database ... 129504 files and directories currently installed.)
Preparing to unpack .../libspatialindex6_1.9.3-1b

In [8]:
# Convert Lat and Long to Congressional Districts
import geopandas as gpd

# convert the store location data into a geodataframe
store_gdf = gpd.GeoDataFrame(store_df, geometry=gpd.points_from_xy(store_df.Longitude, store_df.Latitude))

# read congressional district data into a geodataframe
district_gdf = gpd.read_file("districts114.shp")

# merge the store location data with the congressional district data
merged_gdf = gpd.sjoin(store_gdf, district_gdf, op='within')

# print the merged data
print(merged_gdf)

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3249: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


       Store Number State/Province  Longitude  Latitude  \
11964   3513-125945             AK    -149.78     61.21   
11965   74352-84449             AK    -149.84     61.14   
11966  12449-152385             AK    -149.85     61.11   
11967  24936-233524             AK    -149.89     61.13   
11968    8973-85630             AK    -149.86     61.14   
...             ...            ...        ...       ...   
25567   74385-87621             WY    -105.59     41.32   
25568   73320-24375             WY    -105.56     41.31   
25569  22425-219024             WY    -105.56     41.31   
25570  10849-103163             WY    -109.25     41.58   
25571  10769-102454             WY    -106.94     44.77   

                          geometry  index_right  
11964  POINT (-149.78000 61.21000)          433  
11965  POINT (-149.84000 61.14000)          433  
11966  POINT (-149.85000 61.11000)          433  
11967  POINT (-149.89000 61.13000)          433  
11968  POINT (-149.86000 61.14000)       

In [11]:
merged_gdf.head(100)

,Store Number,State/Province,Longitude,Latitude,geometry,index_right
11964,3513-125945,AK,-149.78,61.21,POINT (-149.78000 61.21000),433
11965,74352-84449,AK,-149.84,61.14,POINT (-149.84000 61.14000),433
11966,12449-152385,AK,-149.85,61.11,POINT (-149.85000 61.11000),433
11967,24936-233524,AK,-149.89,61.13,POINT (-149.89000 61.13000),433
11968,8973-85630,AK,-149.86,61.14,POINT (-149.86000 61.14000),433
...,...,...,...,...,...,...
12086,76606-101463,AL,-85.78,33.61,POINT (-85.78000 33.61000),150
12087,9738-97528,AL,-85.83,33.61,POINT (-85.83000 33.61000),150
12023,24427-237908,AL,-86.80,33.51,POINT (-86.80000 33.51000),154
12024,18675-189964,AL,-86.75,33.56,POINT (-86.75000 33.56000),154
